# Building a Two‑Tier Workflow Compiler
**From Canonical IR to Runtime Simulation**


## Principles

1. **Single canonical DAG IR (compile‑time)** – immutable, no tensors, only symbols.  
2. **Plug‑in registry** – YAML ± decorator entries provide cost formulas & legal transforms.  
3. **Dual capture modes** – static AST + dynamic tracer (+ monkey‑patch fallback) feed the IR.  
4. **Declarative multi‑metric cost model** – FLOPs, memory, qubits, etc., evaluated lazily.

Everything else hangs off these four axioms.



### 1. Existing Runtime Layer (`DirectedEdge`, `Node`, `Graph`, maybe `Process`?)

* `Process` subclasses (**state‑ful, side‑effect‑ful**) implement real algorithms (`update`, `generate_output`, resource accounting)
* `Node` wraps a `Process` instance, tracks inbound/outbound edges and execution status.
* `Network` orchestrates nodes, allocates resources via a `Broker`, and advances simulated time.
* **Data validation** happens inside each `Process` (`validate_data`, `set_expected_input_properties`).



## 2 Canonical IR Layer (`core/ir.py`)

```python
# core/ir.py
from dataclasses import dataclass, field
import networkx as nx
from typing import Dict, Any, Tuple, List
from enum import Enum, auto

class EdgeKind(Enum):
    DATA = auto()
    CTRL = auto()
    MUTATE = auto()

@dataclass
class Node:
    name: str
    op_type: str
    domain: str
    params: Dict[str, Any] = field(default_factory=dict)
    cost:   Dict[str, float] = field(default_factory=dict)
    source_span: Tuple[str,int,int] | None = None

class Graph:
    def __init__(self):
        self.g = nx.DiGraph()
    def add_node(self, n: Node): self.g.add_node(n.name, **n.__dict__)
    def add_edge(self, u: str, v: str, kind: EdgeKind = EdgeKind.DATA):
        self.g.add_edge(u, v, kind=kind.name.lower())
    def topological(self) -> List[dict]:
        import networkx as nx
        return [self.g.nodes[n] for n in nx.topological_sort(self.g)]
```

*No tensors, no resources, no `Process` objects – this is purely declarative.*



## 3 Registry & Cost Engine (`core/registry`, `core/cost.py`)

### 3.1 Kernel & Transform Specs
The **`RegisterSpec`** and **`Signature`** classes you already wrote (see `schema.py` §165‑870) become *validators* that guarantee every YAML registry entry has well‑typed parameters.

### 3.2 Cost Evaluation

```python
# core/cost.py
import numexpr as ne
from core.registry import lookup_kernel

def evaluate(node, user_params):
    spec = lookup_kernel(node.op_type)
    p = {**user_params, **node.params}
    for metric, expr in spec.cost_formula.items():
        node.cost[metric] = float(ne.evaluate(expr, local_dict=p))
```



## 4 Front‑End Capture Stack

### 4.1 Static AST Walker (`frontends.ast_static.py`)
* Uses **libcst** to parse source.
* Matches call‑patterns → `op_type` via a YAML table.
* Extracts shape symbols, populates `Node.params`.

### 4.2 Dynamic FX Tracer (`frontends.fx_dynamic.py`)
* Runs function/module on meta‑tensors.
* Maps `torch.matmul` → `gemm`, etc.
* Resolves runtime tensor shapes.

### 4.3 Monkey‑Patch Fallback (`frontends.hybrid_sampler.py`)
* Wraps functions listed in `wf.yaml.regex_hooks`.
* Emits `Node(op_type="::opaque::<module>.<fn>")` when no other capture succeeds.



## 5 Transform Plug‑ins (`transforms/`)

```python
from core.ir import Graph, Node
from transforms import register_transform

@register_transform('linalg', 'strassen_split')
def strassen_split(node: Node, graph: Graph):
    if node.op_type != 'gemm': return []
    M,N,K = [node.params.get(k) for k in 'MNK']
    if min(M,N,K) < 512: return []
    # build sub‑graph …
    return [subgraph]
```



## 6 Search / Optimisation (brief)

* Generate variants per transform with beam width *k*.
* Solve MILP with OR‑Tools to satisfy resource constraints.
* Return **Graph★** – the chosen compile‑time plan.



## 7 Adapter – Canonical IR → Runtime Network

```python
# core/materialise.py
from qrew.simulation.graph import Node as RtNode, Network as RtNetwork
from proc_lookup import OP_TYPE2PROCESS

def to_runtime(ir_graph):
    id2node = {}
    for n in ir_graph.topological():
        Proc = OP_TYPE2PROCESS[n['op_type']]
        rt = RtNode(process_model=Proc, network_type=RtNode.NETWORK)
        id2node[n['name']] = rt
    for u, v in ir_graph.g.edges():
        id2node[u].insert_output_node(id2node[v])
    return RtNetwork(
        name="compiled_workflow",
        nodes=list(id2node.values()),
        input_nodes=[id2node[n] for n in find_sources(ir_graph)],
        output_nodes=[id2node[n] for n in find_sinks(ir_graph)],
        broker=None,
    )
```



## 8 Case Study – Matrix Inversion via Gaussian Elimination

### 8.1 Compile‑time (symbolic)

1. **Front‑end capture**  
   ```python
   A = sym('M×N')          # symbolic
   ge_plan = invert(A)     # user code calls helper
   ```
   Front‑end emits a canonical graph:

```
find_pivot  ─→  swap_rows  ─→  row_deconstruct
```

2. **Cost evaluation**  
   * `find_pivot`: FLOPs = `2*M`  
   * `swap_rows`: FLOPs = `M`  
   * `row_deconstruct`: FLOPs = `M*N`  
   Total FLOPs formula recorded for optimiser.

3. **Transform search**  
   * Option A – Classic GE (above)  
   * Option B – Use library call `torch.linalg.inv` (`op_type='matrix_inv'`) with cost `O(M^3)`.

4. **Optimiser decision**  
   Under constraint *M ≤ 64* the library call may be faster; otherwise stick to classic GE.

### 8.2 Materialise & Simulate

```python
best = optimise(ir_graph, metric='flops', limits={'mem': 8e9})
runtime_net = to_runtime(best)
runtime_net.run(runtime_net.input_nodes,
                [(Data(matrix, {'Usage': 'Matrix'}),
                 Data(0, {'Usage': 'Column Idx'}))])
result_matrix = runtime_net.output_nodes[0].output_edges[0].data[0].data
```

The **same unit test** you already have passes unchanged because the runtime layer hasn’t moved.



## 9 Interplay of `RegisterSpec` / `Signature` and Data‑Flow Validation

* **Compile‑time** – when a YAML entry is loaded, its `parameters:` field is parsed into `RegisterSpec`s.  
  *Your existing `dtype_parser` checks that each symbol conforms to the dtype lattice described in `fundamentals.md`.*

* **Runtime** – every `Process` builds its own `Signature` (see `GE_RowReduction.signature`).  
  `Node.ensure_ready()` matches inbound `Data` objects by **Usage** and validates shapes/types using `Signature`.

Thus, **validation happens twice**:

| Layer | Goal |
|-------|------|
| Registry load | guarantee that cost formulas and transforms refer to *well‑typed* parameters. |
| Simulation run | guarantee that *concrete* `Data` objects satisfy the expectations of each `Process`. |



## 10 Roadmap Steps (concrete action list)

| Week | Engineering Task |
|------|------------------|
| 1‑2 | Add `core/ir.py`, clone capture logic from `AstDagConverter` ➜ IR. |
| 3‑4 | Port `RegisterSpec`/`Signature` to registry loader; add dtype validation. |
| 5‑6 | Implement AST walker + simple YAML pattern table (`numpy.dot` → `gemm`). |
| 7‑8 | Hook dynamic FX tracer; write adapter `materialise.to_runtime`. |
| 9‑10 | Implement Strassen transform, MILP optimiser, cost formulas for GE kernels. |
| 11‑12 | Compile‑time ↔ runtime integration test: matrix inversion 64×64. |
